# Al-NMC界面解析（混入密度ベース）

界面の断面積で正規化した「界面混入密度」を使用して解析・可視化を行う。

## 解析内容
1. 混入密度の計算（atoms/Å²）
2. 基板材料別のサマリー作成
3. 引張強度と混入密度の2軸比較グラフ
4. 各種分布プロット

## 1. ライブラリ読み込みと設定

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from pathlib import Path

# 日本語フォント対応
try:
    import japanize_matplotlib
except ImportError:
    print("警告: japanize_matplotlibがインストールされていません")

# グラフスタイル設定
sns.set_theme(style="whitegrid")
plt.rcParams['figure.dpi'] = 100

In [ ]:
# ============================================================================
# 設定・定数
# ============================================================================

# 基板の断面積 (Å²)
AREA_MAP = {
    'Al2O3': 13.94673902 * 13.08364686,   # ~182.5 Å²
    'AlF3':  11.76094577 * 12.80689447,   # ~150.6 Å²
    'Al':    16.952258347397713 * 16.952258350558857,  # ~287.4 Å²
}

# 基板を構成する元素（上層への混入をカウント）
SUBSTRATE_ELEMENTS = ['Al']

# カソードを構成する元素（下層への混入をカウント）
CATHODE_ELEMENTS = ['Li', 'Mn', 'Co', 'Ni']

# 可視化の色設定
COLOR_MAP = {
    'Co': '#1f77b4',
    'Mn': '#ff7f0e',
    'Co3O4': '#2ca02c',
    'NiO': '#d62728',
    'Li2O': '#9467bd',
    'Carbon': '#8c564b',
    'graphite': '#e377c2',
    'Li3MnCoNiO6': '#bcbd22',
    'Mn3O4': '#17becf',
}

# フォントサイズ設定
TITLE_FS = 20
LABEL_FS = 18
TICK_FS = 16
LEGEND_FS = 14

# 断面積を確認
print("基板の断面積 (Å²):")
for k, v in AREA_MAP.items():
    print(f"  {k}: {v:.2f}")

## 2. ユーティリティ関数

In [ ]:
def format_chemical_formula(text):
    """
    文字列内の数値をLaTeXの下付き文字形式に変換する。
    例: 'Al2O3' -> 'Al$_{2}$O$_{3}$'
    """
    if not isinstance(text, str) or '$' in text:
        return text
    return re.sub(r'(\d+)', r'$_{\1}$', text)

## 3. 混入密度計算関数

In [ ]:
def calculate_contamination_density(df):
    """
    混入密度を計算する。
    
    計算される列:
        - cross_sectional_area_A2: 断面積 (Å²)
        - substrate_mixed_atoms: 基板から上層への混入原子数
        - cathode_mixed_atoms: カソードから下層への混入原子数
        - total_mixed_atoms: 合計混入原子数
        - substrate_contamination_density: 基板混入密度 (atoms/Å²)
        - cathode_contamination_density: カソード混入密度 (atoms/Å²)
        - total_contamination_density: 全体混入密度 (atoms/Å²)
    """
    df = df.copy()
    
    # 断面積を割り当て
    df['cross_sectional_area_A2'] = df['substrate'].map(AREA_MAP)
    
    if df['cross_sectional_area_A2'].isnull().any():
        unknown = df[df['cross_sectional_area_A2'].isnull()]['substrate'].unique()
        print(f"警告: 未知の基板が見つかりました: {unknown}")
        df['cross_sectional_area_A2'].fillna(0, inplace=True)
    
    # 基板からの混入原子数（上層に移動した基板原子）
    df['substrate_mixed_atoms'] = 0
    for elem in SUBSTRATE_ELEMENTS:
        col = f'{elem}_upper'
        if col in df.columns:
            df['substrate_mixed_atoms'] += df[col].fillna(0)
    
    # カソードからの混入原子数（下層に移動したカソード原子）
    df['cathode_mixed_atoms'] = 0
    for elem in CATHODE_ELEMENTS:
        col = f'{elem}_lower'
        if col in df.columns:
            df['cathode_mixed_atoms'] += df[col].fillna(0)
    
    df['total_mixed_atoms'] = df['substrate_mixed_atoms'] + df['cathode_mixed_atoms']
    
    # 混入密度を計算
    valid_mask = df['cross_sectional_area_A2'] > 0
    
    df['substrate_contamination_density'] = 0.0
    df['cathode_contamination_density'] = 0.0
    df['total_contamination_density'] = 0.0
    
    df.loc[valid_mask, 'substrate_contamination_density'] = (
        df.loc[valid_mask, 'substrate_mixed_atoms'] / df.loc[valid_mask, 'cross_sectional_area_A2']
    )
    df.loc[valid_mask, 'cathode_contamination_density'] = (
        df.loc[valid_mask, 'cathode_mixed_atoms'] / df.loc[valid_mask, 'cross_sectional_area_A2']
    )
    df.loc[valid_mask, 'total_contamination_density'] = (
        df.loc[valid_mask, 'total_mixed_atoms'] / df.loc[valid_mask, 'cross_sectional_area_A2']
    )
    
    return df

In [ ]:
def create_summary_data(df, group_by='substrate'):
    """
    グループごとの平均値を計算してサマリーデータを作成する。
    """
    summary = df.groupby(group_by).agg({
        'tensile_strength_GPa': 'mean',
        'total_contamination_density': 'mean',
        'substrate_contamination_density': 'mean',
        'cathode_contamination_density': 'mean',
    }).reset_index()
    
    return summary


def create_filtered_summary(df, conditions=None):
    """
    条件でフィルタリングしてサマリーを作成する。
    
    Args:
        conditions: フィルタ条件の辞書
            例: {'material': 'Li3MnCoNiO6', 'high_temp_K': 500}
    """
    df_filtered = df.copy()
    
    if conditions:
        for col, val in conditions.items():
            if col in df_filtered.columns:
                df_filtered = df_filtered[df_filtered[col] == val]
    
    if len(df_filtered) == 0:
        print("警告: 指定条件に該当するデータがありません")
        return pd.DataFrame()
    
    return create_summary_data(df_filtered, group_by='substrate')

## 4. 可視化関数

In [ ]:
def plot_comparison_bar(summary_df, title_suffix="", output_path=None):
    """
    引張強度と混入密度の2軸横並び棒グラフを作成する。
    """
    fig, ax1 = plt.subplots(figsize=(10, 6))
    
    materials = summary_df['substrate'].tolist()
    tensile = summary_df['tensile_strength_GPa'].tolist()
    density = summary_df['total_contamination_density'].tolist()
    
    x = np.arange(len(materials))
    width = 0.35
    
    # 左軸: 引張強度（青）
    color1 = '#1f77b4'
    bars1 = ax1.bar(x - width/2, tensile, width, label='Tensile Strength (GPa)', color=color1)
    ax1.set_xlabel('Material Comparison', fontsize=LABEL_FS)
    ax1.set_ylabel('Tensile Strength (GPa)', fontsize=LABEL_FS, color=color1)
    ax1.tick_params(axis='y', labelcolor=color1, labelsize=TICK_FS)
    ax1.tick_params(axis='x', labelsize=TICK_FS)
    ax1.set_xticks(x)
    ax1.set_xticklabels(materials)
    
    # 右軸: 混入密度（オレンジ）
    ax2 = ax1.twinx()
    color2 = '#ff7f0e'
    bars2 = ax2.bar(x + width/2, density, width, label='Total Contamination Density', color=color2)
    ax2.set_ylabel('Total Contamination Density (atoms/Å²)', fontsize=LABEL_FS, color=color2)
    ax2.tick_params(axis='y', labelcolor=color2, labelsize=TICK_FS)
    
    # タイトル
    title = "Material Comparison"
    if title_suffix:
        title += f"\n({title_suffix})"
    plt.title(title, fontsize=TITLE_FS, weight='bold')
    
    # グリッド
    ax1.set_axisbelow(True)
    ax1.yaxis.grid(True, linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    
    if output_path:
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        print(f"保存: {output_path}")
    
    plt.show()

In [ ]:
def plot_density_scatter(df, output_path=None):
    """
    基板混入密度 vs カソード混入密度の散布図を作成する。
    """
    df['substrate_formatted'] = df['substrate'].apply(format_chemical_formula)
    
    plt.figure(figsize=(10, 7))
    sns.scatterplot(
        data=df,
        x='substrate_contamination_density',
        y='cathode_contamination_density',
        style='substrate_formatted',
        hue='material',
        s=200,
        alpha=0.8,
        palette=COLOR_MAP
    )
    plt.title('Substrate vs. Cathode Contamination Density', fontsize=TITLE_FS, weight='bold')
    plt.xlabel('Substrate Contamination Density (atoms/Å²)', fontsize=LABEL_FS)
    plt.ylabel('Cathode Contamination Density (atoms/Å²)', fontsize=LABEL_FS)
    plt.xticks(fontsize=TICK_FS)
    plt.yticks(fontsize=TICK_FS)
    
    handles, labels = plt.gca().get_legend_handles_labels()
    formatted_labels = [format_chemical_formula(label) for label in labels]
    plt.legend(handles, formatted_labels, bbox_to_anchor=(1.02, 1), loc='upper left',
               fontsize=LEGEND_FS, title='Legend')
    
    plt.tight_layout()
    
    if output_path:
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        print(f"保存: {output_path}")
    
    plt.show()

In [ ]:
def plot_density_distribution(df, density_col, title, ylabel, output_path=None):
    """
    混入密度の分布をswarm plotで可視化する。
    """
    df['substrate_formatted'] = df['substrate'].apply(format_chemical_formula)
    
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='substrate_formatted', y=density_col, data=df, 
                color='lightgray', showfliers=False)
    sns.swarmplot(x='substrate_formatted', y=density_col, hue='material', data=df,
                  size=8, palette=COLOR_MAP)
    
    plt.title(title, fontsize=TITLE_FS, weight='bold')
    plt.xlabel('Substrate', fontsize=LABEL_FS)
    plt.ylabel(ylabel, fontsize=LABEL_FS)
    plt.xticks(fontsize=TICK_FS)
    plt.yticks(fontsize=TICK_FS)
    
    handles, labels = plt.gca().get_legend_handles_labels()
    formatted_labels = [format_chemical_formula(label) for label in labels]
    plt.legend(handles, formatted_labels, title='Cathode Material', fontsize=LEGEND_FS-2)
    
    plt.tight_layout()
    
    if output_path:
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        print(f"保存: {output_path}")
    
    plt.show()

In [ ]:
def plot_tensile_distribution(df, output_path=None):
    """
    引張強度の分布を可視化する。
    """
    df['substrate_formatted'] = df['substrate'].apply(format_chemical_formula)
    
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='substrate_formatted', y='tensile_strength_GPa', data=df,
                color='lightgray', showfliers=False)
    sns.swarmplot(x='substrate_formatted', y='tensile_strength_GPa', hue='material', data=df,
                  size=8, palette=COLOR_MAP)
    
    plt.title('Tensile Strength Distribution', fontsize=TITLE_FS, weight='bold')
    plt.xlabel('Substrate', fontsize=LABEL_FS)
    plt.ylabel('Tensile Strength (GPa)', fontsize=LABEL_FS)
    plt.xticks(fontsize=TICK_FS)
    plt.yticks(fontsize=TICK_FS)
    
    handles, labels = plt.gca().get_legend_handles_labels()
    formatted_labels = [format_chemical_formula(label) for label in labels]
    plt.legend(handles, formatted_labels, title='Cathode Material', fontsize=LEGEND_FS-2)
    
    plt.tight_layout()
    
    if output_path:
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        print(f"保存: {output_path}")
    
    plt.show()

## 5. データ読み込みと混入密度計算

In [ ]:
# ★★★ CSVファイルのパスを指定 ★★★
CSV_PATH = "./output/comprehensive_analysis_output/comprehensive_analysis_results.csv"

# データ読み込み
df_raw = pd.read_csv(CSV_PATH)
print(f"データ読み込み完了: {len(df_raw)} 行")
print(f"\n列名: {list(df_raw.columns)}")

In [ ]:
# 混入密度を計算
df = calculate_contamination_density(df_raw)

# 計算結果を確認
print("混入密度計算完了")
print("\n追加された列:")
new_cols = ['cross_sectional_area_A2', 'substrate_mixed_atoms', 'cathode_mixed_atoms',
            'total_mixed_atoms', 'substrate_contamination_density', 
            'cathode_contamination_density', 'total_contamination_density']
for col in new_cols:
    if col in df.columns:
        print(f"  {col}: min={df[col].min():.4f}, max={df[col].max():.4f}, mean={df[col].mean():.4f}")

## 6. 基板別サマリー

In [ ]:
# 全データの基板別サマリー
summary_all = create_summary_data(df, group_by='substrate')
print("=== 基板別サマリー（全データ平均） ===")
print(summary_all.round(4).to_string(index=False))

In [ ]:
# カソード材料別サマリー
if 'material' in df.columns:
    summary_material = df.groupby('material').agg({
        'tensile_strength_GPa': 'mean',
        'total_contamination_density': 'mean',
    }).reset_index()
    print("=== カソード材料別サマリー ===")
    print(summary_material.round(4).to_string(index=False))

## 7. 2軸比較棒グラフ（引張強度 vs 混入密度）

In [ ]:
# 全データ平均の比較グラフ
plot_comparison_bar(summary_all, title_suffix="All Data Average")

In [ ]:
# 条件別の比較グラフ（例）
# ★★★ 条件を変更して実行 ★★★

conditions = {
    # 'material': 'Li3MnCoNiO6',  # カソード材料
    # 'high_temp_K': 500,          # 温度
    # 'pressure_GPa': 0.01,        # 圧力
}

if conditions:
    summary_filtered = create_filtered_summary(df, conditions)
    if len(summary_filtered) > 0:
        title_suffix = ", ".join([f"{k}: {v}" for k, v in conditions.items()])
        plot_comparison_bar(summary_filtered, title_suffix=title_suffix)
else:
    print("条件を指定してください")

## 8. 混入密度散布図

In [ ]:
plot_density_scatter(df)

## 9. 分布プロット

In [ ]:
# 全体混入密度の分布
plot_density_distribution(
    df, 
    'total_contamination_density',
    'Total Contamination Density Distribution',
    'Total Contamination Density (atoms/Å²)'
)

In [ ]:
# カソード混入密度の分布
plot_density_distribution(
    df,
    'cathode_contamination_density',
    'Cathode Contamination Density Distribution',
    'Cathode Contamination Density (atoms/Å²)'
)

In [ ]:
# 基板混入密度の分布
plot_density_distribution(
    df,
    'substrate_contamination_density',
    'Substrate Contamination Density Distribution',
    'Substrate Contamination Density (atoms/Å²)'
)

In [ ]:
# 引張強度の分布
plot_tensile_distribution(df)

## 10. 結果の保存

In [ ]:
# 出力ディレクトリ
OUTPUT_DIR = Path("./output/contamination_analysis")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# サマリーCSVを保存
summary_all.to_csv(OUTPUT_DIR / "substrate_summary.csv", index=False)
print(f"保存: {OUTPUT_DIR / 'substrate_summary.csv'}")

# 詳細データを保存
df.to_csv(OUTPUT_DIR / "contamination_density_results.csv", index=False)
print(f"保存: {OUTPUT_DIR / 'contamination_density_results.csv'}")

In [ ]:
# グラフを保存
plot_comparison_bar(summary_all, "All Data Average", OUTPUT_DIR / "comparison_bar.png")
plot_density_scatter(df, OUTPUT_DIR / "density_scatter.png")
plot_density_distribution(df, 'total_contamination_density', 
                          'Total Contamination Density', 'Density (atoms/Å²)',
                          OUTPUT_DIR / "total_density_distribution.png")
plot_tensile_distribution(df, OUTPUT_DIR / "tensile_distribution.png")

print(f"\n全ての結果を {OUTPUT_DIR} に保存しました")